# Week 1 
# Intro to working with neural nets in keras


Before running this the first time, please make sure you have (1) installed the necessary Python tools and libraries, and (2) activated your enviromment!

Please see instructions for this under "Instructions for Setting Up & Using Python & Jupyter" in [moodle](https://moodle.arts.ac.uk/course/view.php?id=71166).

In [1]:
import tensorflow as tf
import keras

keras.__version__ #print out current keras version

'2.12.0'

In [2]:
tf.__version__ #print out current tensorflow version

'2.12.0'

In [3]:
tf.config.experimental.list_physical_devices('GPU') #Do we have access to a GPU?

[]

In [4]:
# Now do your imports
import numpy as np
from keras.models import Sequential #base keras model
from keras.layers import Dense, Activation #dense = fully connected layer
from tensorflow.keras.optimizers.legacy import SGD #this is just tensorflow.keras.optimizers on earlier versions of tf
import h5py # for saving a trained network

## Part 1: A very, very simple example

Can we train a single, 1-input neuron to learn the function output = 5 * input ? 

In [5]:
#Let's make a training dataset with 3 examples
x_train = np.array([[2], [1], [-3]]) #Input values for each of our 3 training examples
y_train = np.array([10, 5, -15]) #Output values (aka "targets") for each of our 3 training examples

In [6]:
# Create the network using keras
num_neurons = 1
model = Sequential() #the basic keras model class
model.add(Dense(num_neurons, input_dim = 1)) #add a "layer" of 1 neuron, which has 1 input value. This will compute a weighted sum, with no additional activation function.
model.summary() #print out info about this network

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [7]:
#Set some training parameters
#use stochastic gradient descent (pretty standard optimisation method) with a learning rate of 0.1 (why not?)
sgd = SGD(learning_rate=0.1) #Note that in previous versions the "learning_rate" parameter was titled "lr" instead

#optimise the mean squared error (i.e., the mean of the squared difference between the model's output and the training target, for each training example)
#    This is a fairly standard choice when predicting a real value (as opposed to a binary class of 0 or 1)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [8]:
# Run the current model on each of the training examples (i.e., using the inputs to calculate the neuron output)
model.predict(x_train) #outputs garbage as it's not trained yet: It's computing output = w1 * input + w0 using random values of w0 and w1

1/1 [==============================] - 0s 78ms/step


array([[-3.1124496],
       [-1.5562248],
       [ 4.6686745]], dtype=float32)

In [9]:
#train the model!
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 160ms/step - loss: 200.5924
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8915
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.0040
Epoch 4/10
1/1 [==============================] - 0s 3ms/step - loss: 1.7611e-05
Epoch 5/10
1/1 [==============================] - 0s 5ms/step - loss: 7.8503e-08
Epoch 6/10
1/1 [==============================] - 0s 6ms/step - loss: 3.4379e-10
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 1.5916e-12
Epoch 8/10
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00
Epoch 9/10
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00
Epoch 10/10
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00


In [10]:
#Now run it:
model.predict(x_train) #Does it produce values similar to y_train? It should...

1/1 [==============================] - 0s 17ms/step


array([[ 10.],
       [  5.],
       [-15.]], dtype=float32)

In [11]:
#we can (and should) run it on some new data, too... 
new_values = np.array([[-10], [0], [5]]) # 3 new "data points"

#Does it output 5 * x for each new value x ?
model.predict(new_values) #be careful to read this using scientific notation ;)

1/1 [==============================] - 0s 18ms/step


array([[-5.0000000e+01],
       [ 1.6268558e-08],
       [ 2.5000000e+01]], dtype=float32)

In [12]:
#Let's look at the model's weights! We'll see (1) the weight for the input, and (2) the weight for the bias
model.get_weights()

[array([[5.]], dtype=float32), array([1.6268558e-08], dtype=float32)]

In [14]:
# Perhaps try with a new dataset in which our training example output (target) values are np.array([11, 6, -14])?
y_train_new = np.array([11, 6, -14])
model = Sequential() #the basic model class
model.add(Dense(num_neurons, input_dim = 1)) #add a "layer" of 1 neuron, which has 1 input value. This will compute a weighted sum, with no additional activation function.
model.summary()
sgd = SGD(learning_rate=0.1) 
model.compile(loss='mean_squared_error', optimizer=sgd)
model.fit(x_train, y_train_new, epochs=15)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
1/1 [==============================] - 0s 119ms/step - loss: 138.2414
Epoch 2/15
1/1 [==============================] - 0s 9ms/step - loss: 1.2500
Epoch 3/15
1/1 [==============================] - 0s 7ms/step - loss: 0.4123
Epoch 4/15
1/1 [==============================] - 0s 6ms/step - loss: 0.2622
Epoch 5/15
1/1 [==============================] - 0s 6ms/step - loss: 0.1678
Epoch 6/15
1/1 [==============================] - 0s 7ms/step - loss: 0.1074
Epoch 7/15
1/1 [==============================] - 0s 4ms/step - loss: 0.0687
Epoch 8/15
1/1 [=============================

In [15]:
model.get_weights() #Notice weight is 5 and bias is now 1 (approximately), encoding the function output = input * 5 + 1

[array([[5.]], dtype=float32), array([0.9648157], dtype=float32)]